In [ ]:
import os, gc, time
username = os.getenv('USER')
import numpy as np
import pandas as pd
import xarray as xr

import dask
dask.config.set(**{'array.slicing.split_large_chunks': False})

from dscim import ProWaiter

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
discount_types = ["euler_ramsey", "constant"]
menu_options = ["risk_aversion", "adding_up", ]
AR_list = [6]
pulse_years = [2020]
# pulse_years = range(2020, 2081, 10)
weitzman_values = [x / 10.0 for x in range(1, 11, 1)] + [0.25, 0.01, 0.001, 0.0001] 

In [ ]:
sectors =  ["energy_USA"] # "mortality_USA", "agriculture_USA", "energy_USA", 
eta_rhos = { 
    2.0 : 0.0,
    1.016010255 : 9.149608e-05,
    1.244459066 : 0.00197263997,
    1.421158116 : 0.00461878399,
    1.567899395 : 0.00770271076
}

combos = [(s, p, d, m, ar, eta) 
          for p in pulse_years
          for s in sectors  
          for d in discount_types 
          for m in menu_options
          for ar in AR_list
          for eta in eta_rhos.keys()]

In [ ]:
n_combos = len(combos)

while n_combos > 0: 
    combo = combos[0]
    print("=========================================================")
    print(combo)
    sector = combo[0]
    pulse_year = combo[1]
    discount_type = combo[2]
    menu_option = combo[3]
    AR = combo[4]
    eta = combo[5]
    w = ProWaiter(path_to_config=f'/home/{username}/repos/integration/configs/USA_SCC_ssps.yaml')
    
    save_path= f'/mnt/CIL_integration/USA_SCC/{sector}/{pulse_year}/unmasked/'
    
    kwargs = {'discounting_type' : discount_type,
              'sector': sector,
              'ce_path' : f'/shares/gcp/integration/CE_library_epa_vsl_bc39/{sector}/',
              'save_path' : save_path,
              'weitzman_parameter' : weitzman_values,
              'pulse_year' : pulse_year,
              'eta': eta,
              'rho': eta_rhos[eta]
             }

    if 'CAMEL' in sector:
        kwargs.update({'damage_function_path' : f"/mnt/CIL_integration/damage_function_library/damage_function_library_USA_SCC/{sector}",
                      })

    menu_item = w.menu_factory(menu_key=menu_option,
                                           sector=sector,
                                           kwargs=kwargs
                                          )
    
    menu_item.order_plate('damage_function')

    combos.remove(combo)
    n_combos = len(combos)
    print(f"remaining combos: {n_combos}")

